In [1]:
import cv2
import numpy as np

In [2]:
sift = cv2.SIFT_create()

In [3]:
import glob

files = glob.glob('./traindata/*')

In [68]:
img = cv2.imread('./testData/dog.3144.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
kp, des = sift.detectAndCompute(gray, None)
kpImg = cv2.drawKeypoints(gray, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) 
cv2.imwrite('./report/dog3144kp.jpg', kpImg)


True

In [69]:
img = cv2.imread('./testData/cat.4433.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
kp, des = sift.detectAndCompute(gray, None)
kpImg = cv2.drawKeypoints(gray, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS) 
cv2.imwrite('./report/cat4433kp.jpg', kpImg)

True

In [10]:
img = cv2.imread('./traindata/cat.0.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
kp, des = sift.detectAndCompute(gray, None)
des.shape

(87, 128)

In [19]:
# ファイルを移動
import shutil
import os

for file in glob.glob('./train_19000/*'):
    print(file)
    fileString = file.split('.')
    if (int(fileString[-2]) >= 3000 and int(fileString[-2]) < 5000):
        shutil.copy(file, './testData/')

./train_19000/dog.8011.jpg
./train_19000/cat.5077.jpg
./train_19000/dog.7322.jpg
./train_19000/cat.10151.jpg
./train_19000/cat.3406.jpg
./train_19000/cat.4369.jpg
./train_19000/cat.7660.jpg
./train_19000/dog.5535.jpg
./train_19000/cat.8553.jpg
./train_19000/cat.9895.jpg
./train_19000/dog.3144.jpg
./train_19000/dog.11102.jpg
./train_19000/cat.6218.jpg
./train_19000/dog.11664.jpg
./train_19000/dog.3622.jpg
./train_19000/cat.12020.jpg
./train_19000/dog.5253.jpg
./train_19000/cat.7106.jpg
./train_19000/cat.8235.jpg
./train_19000/dog.9469.jpg
./train_19000/cat.3360.jpg
./train_19000/cat.11529.jpg
./train_19000/cat.10637.jpg
./train_19000/dog.6982.jpg
./train_19000/dog.8777.jpg
./train_19000/dog.7444.jpg
./train_19000/cat.5711.jpg
./train_19000/cat.3374.jpg
./train_19000/dog.6996.jpg
./train_19000/cat.10623.jpg
./train_19000/dog.8763.jpg
./train_19000/cat.5705.jpg
./train_19000/dog.7450.jpg
./train_19000/dog.12379.jpg
./train_19000/dog.11670.jpg
./train_19000/dog.4159.jpg
./train_19000/dog.3

In [4]:
from sklearn.cluster import KMeans
def create_visualWords(filePath, vectorSize, saveFileName):
    features = []
    files = glob.glob(filePath)
    # SIFT特徴量を取り出す
    for file in files:
        img = cv2.imread(file)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray, None)
        features.extend(des)
    features = np.array(features)
    print("done to calculate sift")
    # 特徴点をクラスタリングしてvisual word を作成する
    visual_words = KMeans(n_clusters=vectorSize).fit(features).cluster_centers_
    np.save(saveFileName, visual_words)
    return visual_words

(1553843, 128)

In [65]:
visual_words.shape

(128, 128)

In [5]:
def calc_featureVec(img, visual_words):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray, None)
    vector = np.zeros(len(visual_words))
    for d in des:
        vector[((visual_words - d)**2).sum(axis=1).argmin()] += 1
    d = np.linalg.norm(vector)
    x = vector
    return x


In [35]:
def createVectors(folderPath, visual_words):
    X = []
    y = []
    num = []
    files = glob.glob(folderPath)
    # SIFT特徴量を取り出す
    for file in files:
        img = cv2.imread(file)
        vector = calc_featureVec(img, visual_words)
        X.append(vector)
        num.append(file.split('.')[-2])
        # add ans
        ans = file.split('.')[-3].split('/')[-1]
        if (ans == 'cat'):
            y.append(0)
        else:
            y.append(1)
    return X, y, num

Naive Bayes

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB



In [37]:
# visual words の作成
visual_words = create_visualWords('./traindata/*', 16, './train3000_16')

In [29]:
# 読み込み
visual_words = np.load('./train3000_16.npy')

In [30]:
# ベクトル生成
X_train, y_train, num_train = createVectors('./trainData/*', visual_words)

In [32]:
X_train

[array([37., 56., 58., 13., 31., 21., 69., 70., 20., 94., 19., 32., 49.,
         0., 15., 61.]),
 array([ 88.,  66.,  92.,  43., 117., 125.,  71.,  68., 215., 114.,  70.,
         73., 142.,  34.,  73., 104.]),
 array([ 33.,  85.,  39.,  22.,  47.,  51.,  49., 106.,  56.,  33.,  39.,
         26.,  38.,  21.,  18.,  42.]),
 array([ 16.,  49.,  29.,  16., 131., 139.,  40.,  64.,  17.,   5.,  55.,
         13.,  23., 150.,  20.,  25.]),
 array([11., 10., 11.,  6., 19., 19., 12.,  6.,  6.,  6.,  3.,  7.,  4.,
         8., 10., 13.]),
 array([366.,  55., 175.,  54., 241., 214.,  58.,  43., 182., 405., 133.,
        142., 244., 386., 197., 173.]),
 array([13., 44., 23.,  8., 31., 21., 15., 37.,  7., 10., 19.,  7., 13.,
        25., 23., 16.]),
 array([12., 50., 19.,  7., 24., 24., 18., 39., 11., 17., 17.,  6., 14.,
        15., 16., 14.]),
 array([ 8., 75., 13.,  2.,  6.,  9.,  9., 47.,  3., 14.,  8.,  6.,  9.,
        26., 11., 11.]),
 array([61., 27., 51., 20., 14., 19., 43., 31., 68., 4

In [33]:
y_train

[0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,


In [34]:
num_train

['1273',
 '2377',
 '2159',
 '228',
 '561',
 '2273',
 '2756',
 '1228',
 '2692',
 '2884',
 '2637',
 '2051',
 '1406',
 '1998',
 '1059',
 '2769',
 '1930',
 '277',
 '1973',
 '2543',
 '1361',
 '809',
 '2892',
 '839',
 '600',
 '2091',
 '2485',
 '2524',
 '2129',
 '2066',
 '2064',
 '860',
 '2448',
 '398',
 '672',
 '1265',
 '306',
 '18',
 '790',
 '1278',
 '203',
 '142',
 '570',
 '182',
 '2263',
 '89',
 '1635',
 '817',
 '2944',
 '2923',
 '1952',
 '607',
 '2106',
 '1692',
 '746',
 '775',
 '2407',
 '1906',
 '1518',
 '659',
 '228',
 '401',
 '1109',
 '1633',
 '752',
 '1116',
 '877',
 '1350',
 '1588',
 '1132',
 '70',
 '2258',
 '1858',
 '510',
 '2736',
 '845',
 '1927',
 '2911',
 '1448',
 '387',
 '1697',
 '2369',
 '878',
 '2634',
 '136',
 '1949',
 '2492',
 '829',
 '380',
 '230',
 '2209',
 '151',
 '436',
 '803',
 '846',
 '760',
 '1745',
 '2694',
 '2363',
 '1415',
 '139',
 '476',
 '2915',
 '719',
 '369',
 '2924',
 '991',
 '2615',
 '1378',
 '555',
 '1544',
 '2719',
 '2233',
 '1398',
 '1426',
 '1249',
 '184

In [36]:
X_test, y_test, num_test = createVectors('./testData/*', visual_words)

In [37]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 3)
gnb = GaussianNB()

In [69]:
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [38]:
g = gnb.fit(X_train, y_train)

In [23]:
from joblib import dump, load

dump(g, './gnb.joblib')

['./gnb.joblib']

In [39]:
y_pred = g.predict(X_test)

In [40]:
print("Number of mislabeled points out of a total %d points : %d"% (np.array(X_test).shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 4000 points : 1682


In [41]:
num_test

['3406',
 '4369',
 '3144',
 '3622',
 '3360',
 '3374',
 '4159',
 '3636',
 '3150',
 '3412',
 '3178',
 '4617',
 '4355',
 '4433',
 '4171',
 '4165',
 '4427',
 '3348',
 '4341',
 '4603',
 '3187',
 '4830',
 '3839',
 '3193',
 '4824',
 '4396',
 '3805',
 '3811',
 '4818',
 '4382',
 '3162',
 '3420',
 '4429',
 '3346',
 '3604',
 '3610',
 '3352',
 '3434',
 '3176',
 '4619',
 '4373',
 '4631',
 '4157',
 '3638',
 '4415',
 '4401',
 '4143',
 '4625',
 '3408',
 '4367',
 '4816',
 '3385',
 '3391',
 '4802',
 '4398',
 '3823',
 '4194',
 '3837',
 '4180',
 '3189',
 '4779',
 '3016',
 '3554',
 '3232',
 '3770',
 '3764',
 '3226',
 '4549',
 '3540',
 '3002',
 '4207',
 '3568',
 '4745',
 '3994',
 '4023',
 '4561',
 '4575',
 '3980',
 '3758',
 '4037',
 '4751',
 '4989',
 '4213',
 '3597',
 '4962',
 '4976',
 '3583',
 '4786',
 '3957',
 '3943',
 '4792',
 '3572',
 '3030',
 '4987',
 '3756',
 '4039',
 '3214',
 '3200',
 '3742',
 '3024',
 '4993',
 '4209',
 '3566',
 '4763',
 '4221',
 '3228',
 '4547',
 '4005',
 '4011',
 '4553',
 '4235',
 

In [63]:
y_test[13]

0

In [64]:
y_pred[13]

1

In [65]:
num_test[13]

'4433'

In [26]:
y_pred, np.array(y_test)

(array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 1, ..., 1, 1, 0]))

In [27]:
y_test

[0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,


In [40]:
# ベクトル生成
X_, y_ = createVectors('./test/*', visual_words)

In [41]:
y_pred_ = gnb.fit(X_train, y_train).predict(X_)

In [43]:
y_pred_

array([0])